In [26]:
import pandas as pd
import talib as ta
import numpy as np
import json
import os

pd.options.display.max_columns = 34
pd.options.display.float_format = '{:.3f}'.format

In [27]:
import warnings
warnings.filterwarnings("ignore")

In [28]:
from pathlib import Path
from sys import path

notebook_path = Path.cwd()
SITE = notebook_path.parent
path.append(str(SITE.absolute()))
from libs.helpers import *

In [29]:
current_dir = os.getcwd()
current_dir

'/home/hung/Documents/PROJECT/gapping/AQ_Partners/bot'

In [30]:

file_path = os.path.join(current_dir, '../../data/investing/currencies/results/DXY.json')

with open(file_path, 'r') as file:
    data = json.load(file)

dxy = pd.DataFrame(data)
dxy = dxy[['day_', 'close_', 'open_', 'high_', 'low_']]
dxy.rename(columns={'day_': 'date', 'close_': 'c', 'open_': 'o', 'high_': 'h', 'low_': 'l'}, inplace=True)
dxy.set_index('date', inplace=True)
dxy.index = pd.to_datetime(dxy.index)
# dxy.index
for column in dxy.columns:
    dxy[column] = pd.to_numeric(dxy[column], errors='coerce')

dxy = dxy[::-1]
dxy['chg'] = dxy['c'].pct_change() * 100
# dxy['chg'] = (dxy['c'] - dxy['c'].shift(1)) * 100 / dxy['c'].shift(1)
# dxy.info()
dxy.dropna(inplace=True)
# dxy

In [31]:
# dxy = calculate_close_to_close_volatility(dxy, 'c', [5,], [21,])
# dxy = calculate_parkinson_volatility(dxy, 'h', 'l', [5,], [21,])
# dxy = calculate_garman_klass_volatility(dxy, 'h', 'l', 'c', 'o', [5,], [21,])
# dxy = calculate_hodges_tompkins_volatility(dxy, 'c', [5,], [21,])
# dxy = calculate_rogers_satchell_volatility(dxy, 'h', 'l', 'c', 'o', [5,], [21,])
# dxy = calculate_yang_zhang_volatility(dxy, 'h', 'l', 'c', 'o', [5,], [21,])

In [32]:
# functions = ['mean', 'var', 'std', 'skew', 'kurt']
# dxy = add_rolling_functions(dxy, ['chg',], ['5D', '21D'], functions)

In [33]:
# # dxy = add_percentage_change(dxy, 'c', ['W', 'M', '3M', 'YTD', 'Y'])
# dxy = add_percentage_change(dxy, 'c', ['W', 'M'])

In [34]:
# indicators = {
#     'EMA': {
#         'time_periods': [5, 8, 13, 21, 34],
#         'input_columns': 'c'
#     },
#     'TRANGE': {
#         'time_periods': "",
#         'input_columns': ('h', 'l', 'c')
#     }
# }
# dxy = add_technical_indicators(dxy, indicators)

In [35]:
# dxy.tail()

In [36]:
weekly_dxy = dxy.resample('W').agg({'c': 'last', 'o': 'first', 'h': 'max', 'l': 'min', 'chg': 'sum'})

weekly_dxy['WLowMax'] = dxy.groupby(pd.Grouper(freq='W'))['l'].max()
weekly_dxy['WHighMin'] = dxy.groupby(pd.Grouper(freq='w'))['h'].min()
weekly_dxy['WM_Diff'] = weekly_dxy['WLowMax'] - weekly_dxy['WHighMin']


weekly_dxy['WStat'] = weekly_dxy.apply(lambda row: 1 if row['chg'] > 0 else 0, axis=1)
weekly_dxy.tail(10)

,c,o,h,l,chg,WLowMax,WHighMin,WM_Diff,WStat
date,,,,,,,,,
2024-02-25,103.940,104.280,104.410,103.430,-0.346,104.140,104.050,0.090,0
2024-03-03,103.860,103.960,104.290,103.610,-0.076,103.840,103.920,-0.080,0
2024-03-10,102.710,103.890,103.960,102.360,-1.111,103.730,102.900,0.830,0
2024-03-17,103.430,102.740,103.490,102.640,0.700,103.300,102.930,0.370,1
2024-03-24,104.410,103.450,104.500,103.170,0.947,104.370,103.650,0.720,1
2024-03-31,104.500,104.410,104.730,104.010,0.086,104.430,104.340,0.090,1
2024-04-07,104.360,104.480,105.100,103.920,-0.131,104.680,104.260,0.420,0
2024-04-14,105.960,104.360,106.110,103.880,1.530,105.920,104.200,1.720,1
2024-04-21,106.110,105.960,106.520,105.740,0.143,106.080,106.140,-0.060,1


In [37]:
monthly_dxy = weekly_dxy.resample('M').agg({'c': 'last','o': 'first','h': 'max','l': 'min', 'chg': 'sum'})

# monthly_dxy['Month'] = monthly_dxy.index.strftime('%b')

monthly_dxy['MLowMax'] = weekly_dxy.groupby(pd.Grouper(freq='M'))['l'].max()
monthly_dxy['MHighMin'] = weekly_dxy.groupby(pd.Grouper(freq='M'))['h'].min()

monthly_dxy['MM_Diff'] = monthly_dxy['MLowMax'] - monthly_dxy['MHighMin']
monthly_dxy['MStat'] = monthly_dxy.apply(lambda row: 1 if row['chg'] > 0 else 0, axis=1)
monthly_dxy.tail(10)

# monthly_dxy.to_csv(f'data/{filename}_OK_Monthly.csv')

,c,o,h,l,chg,MLowMax,MHighMin,MM_Diff,MStat
date,,,,,,,,,
2023-07-31,101.620,102.870,103.570,99.580,-1.226,102.320,101.190,1.130,0
2023-08-31,104.080,101.890,104.450,101.740,2.399,103.010,102.840,0.170,1
2023-09-30,105.580,104.190,105.780,102.940,1.441,104.670,104.360,0.310,1
2023-10-31,106.560,105.570,107.350,105.360,0.939,105.970,106.670,-0.700,1
2023-11-30,103.400,106.580,107.110,103.180,-2.983,104.940,104.210,0.730,0
2023-12-31,101.380,103.420,104.260,100.620,-1.950,103.060,101.770,1.290,0
2024-01-31,103.430,101.420,103.820,101.340,2.013,102.770,102.760,0.010,1
2024-02-29,103.940,103.470,104.980,102.900,0.502,103.940,104.040,-0.100,1
2024-03-31,104.500,103.960,104.730,102.360,0.547,104.010,103.490,0.520,1
